In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torchvision
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import seaborn as sns
from torchvision import datasets as datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

from model_resnet import ResNet18, ResNet50, ResNet50_dropblock, ResNet50_dropchannel, ResNet50_droplayer
from datasets import get_data



In [4]:
class Args:
    batch_size = 256
    train_batch_size = 5000
    lr = 1e-2
    gamma = 0.99
    log_interval = 100
    epochs = 20

args = Args()



In [5]:
dataset = 'svhn'
loaders = get_data(dataset)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [6]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [7]:
train_loader = loaders['train']
test_loader = loaders['valid']
# model = Net()
# model = ResNet50_dropblock(input_size=3, dropout_rate=0.2)
# model = ResNet50_dropchannel(input_size=3, dropout_rate=0.2)
model = ResNet50_droplayer(input_size=3, dropout_rate=0.2)



In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
optimizer = torch.optim.Adadelta(model.parameters(), lr=args.lr)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/73257 (0%)]	Loss: 2.594809
Train Epoch: 1 [51200/73257 (69%)]	Loss: 2.405619

Test set: Average loss: 2.2007, Accuracy: 5468/26032 (21%)

Train Epoch: 2 [0/73257 (0%)]	Loss: 2.309448
Train Epoch: 2 [51200/73257 (69%)]	Loss: 2.196462

Test set: Average loss: 2.0347, Accuracy: 7419/26032 (28%)

Train Epoch: 3 [0/73257 (0%)]	Loss: 2.162640
Train Epoch: 3 [51200/73257 (69%)]	Loss: 2.024504

Test set: Average loss: 1.9147, Accuracy: 8739/26032 (34%)

Train Epoch: 4 [0/73257 (0%)]	Loss: 2.318858


In [ ]:
predictions = []
labels = []
probs = []
with torch.no_grad():
    for x_batch, y_batch in  test_loader:
        logits = model(x_batch.cuda())
        probs.append(torch.max(torch.softmax(logits, dim=-1), dim=-1).values.cpu().numpy())
        predictions.append(torch.argmax(logits, dim=-1).cpu().numpy())
        labels.append(y_batch.numpy())
labels = np.concatenate(labels)
predictions = np.concatenate(predictions, axis=-1)
probs = np.concatenate(probs, axis=-1)


In [ ]:
errors = labels != predictions


In [ ]:
uq = 1 - probs


In [ ]:
T = 200

# model.train()
model.eval()
predictions = []
for t in range(T):
    if (t+1) % 10  == 0:
        print(t)
    probs = []
    for x_batch, y_batch in  test_loader:
        with torch.no_grad():
            logits = model(x_batch.cuda(), dropout_mask=True)
        probs.append(torch.softmax(logits, dim=-1).cpu().numpy())
    predictions.append(np.concatenate(probs, axis=0))


In [ ]:
predictions = np.array(predictions)
predictions.shape


In [ ]:
def entropy(preds):
    return -np.mean((np.sum(np.log(preds) * preds, axis=-1)), axis=0)

def std_uq(preds):
    idxs = np.argmax(np.mean(preds, axis=0), axis=-1)
    return np.std(preds[:, np.arange(len(idxs)), idxs], axis=0)

def bald(preds):
    means = np.mean(preds, axis=0)
    return - np.sum(np.log(means) * means, axis=-1) + entropy(preds)

def max_with_std(preds):
    stds = np.std(preds, axis=0)
    means = np.mean(preds, axis=0)
    mean_std = means - stds
    return 1 - np.max(mean_std, axis=-1)


acquisition = bald
uq_mc = acquisition(predictions)


In [ ]:
plt.style.use(['seaborn'])
fpr, tpr, _ = roc_curve(errors, uq)
plt.plot(fpr, tpr, label=f'max prob uncertainty')

fpr, tpr, _ = roc_curve(errors, uq_mc)
plt.plot(fpr, tpr, label=f'mc dropout ({acquisition.__name__})')
plt.title(f"Resnet50, Dropchannels, {dataset}")
plt.legend()

In [ ]:
aucs = []
nums =  np.linspace(2, 500, 50)
max_score = 0
for num in nums:
    uq_sub = acquisition(predictions[:int(num)])
    score = roc_auc_score(errors, uq_sub)
    if score > max_score:
        i = num
        max_score = score
    aucs.append(score)
plt.plot(nums, aucs)
print(i)

In [ ]:

stds = np.std(predictions, axis=0)
means = np.mean(predictions, axis=0)


In [ ]:
df = pd.DataFrame({'errors':errors, 'stds': stds[:, 0]})
plt.figure(figsize=(10, 8))
sns.displot(df, x='stds', bins=20)
plt.title("Mc dropout std distribution")


In [ ]:
uq_1 = entropy(predictions)
uq_2 = max_with_std(predictions)
plt.scatter(uq_1, uq_2)

In [ ]:
mean_std = means - stds
args_1 = np.argmax(means, axis=-1)
args_2 = np.argmax(mean_std, axis=-1)
np.sum(args_1 != args_2)

In [ ]:
diff_idx = args_1 != args_2

In [ ]:
diff_idx


In [ ]:
# plt.rcParams['axes.facecolor'] = 'white'

plt.style.use(['seaborn'])
plt.title("Points with diffent max classes by mean prob and mean-std prob")
plt.xlabel('Max prob')
plt.ylabel('Max prob for mean std')
plt.scatter(np.max(means[diff_idx], axis=-1), np.max(mean_std[diff_idx], axis=-1))


In [ ]:
images = []
labels = []

for x_batch, y_batch in test_loader:
    images.append(x_batch)
    labels.append(y_batch)



In [ ]:
imgs_t = torch.cat(images, dim=0)


In [ ]:
imgs = np.concatenate(images, axis=0)
img_labels = np.concatenate(labels, axis=0)

In [ ]:
def plt_img(img):
    img = np.rollaxis(img, 0, 3)
    plt.imshow(img)
    plt.show()

plt_img(imgs[0])

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [ ]:
import torchvision
plt.style.use(['default'])

plt.figure(figsize=(10, 4))
plt.title("Common images")
imshow(torchvision.utils.make_grid(imgs_t[:8]))
plt.show()

plt.figure(figsize=(10, 4))
plt.title("Strange images")
imshow(torchvision.utils.make_grid(imgs_t[diff_idx][:8]))

In [ ]:
from sklearn.manifold import TSNE

nums = 1_000
data_1000 = imgs[:nums].reshape(nums, -1)
labels_1000 = img_labels[:nums]

model = TSNE(n_components=2, random_state=0)
# configuring the parameteres
# the number of components = 2
# default perplexity = 30
# default learning rate = 200
# default Maximum number of iterations for the optimization = 1000
tsne_data = model.fit_transform(data_1000)
# creating a new data frame which help us in ploting the result data
tsne_data = np.vstack((tsne_data.T, labels_1000)).T
tsne_df = pd.DataFrame(data=tsne_data, columns=('dim_1', 'dim_2', 'label'))
# Ploting the result of tsne
sns.FacetGrid(tsne_df, hue='label', size=6).map(plt.scatter, 'dim_1', 'dim_2').add_legend()
plt.show()



In [ ]:
imgs[:1000, 0].shape